In [1]:
import pandas as pd 

In [2]:
def read_csv(file_path: str, sep: str = ",") -> pd.DataFrame:
    """
    Đọc dữ liệu từ file CSV và trả về DataFrame.

    Parameters:
        file_path (str): Đường dẫn đến file CSV cần đọc.
        sep (str): Ký tự phân tách giữa các cột, mặc định là ','.Parameters

    Returns:
        pd.DataFrame: Dữ liệu được đọc vào từ file CSV.
    """
    try:
        df = pd.read_csv(file_path, sep=sep, encoding="utf-8")
        return df
    except FileNotFoundError:
        print(f" Không tìm thấy file tại: {file_path}")
    except Exception as e:
        print(f"Lỗi khi đọc file CSV: {e}")
    return pd.DataFrame()


def to_csv(df: pd.DataFrame, file_path: str) -> bool:
    """
    Lưu DataFrame thành file CSV sau khi làm sạch dữ liệu.

    Parameters:
        df (pd.DataFrame): DataFrame cần lưu.
        file_path (str): Đường dẫn lưu file CSV.

    Returns:
        bool: True nếu lưu thành công, False nếu gặp lỗi.
    """
    try:
        df.to_csv(file_path, index=False, encoding="utf-8")
        print(f"File đã được lưu tại: {file_path}")
        return True
    except FileNotFoundError:
        print(f"Không tìm thấy đường dẫn: {file_path}")
    except Exception as e:
        print(f"Lỗi khi lưu file CSV: {e}")
    return False

In [3]:
def extract_rating_features(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
    """
    Tách điểm trung bình và số lượt đánh giá từ cột 'bus_rating'.

    Ví dụ:
        "4.7 (123)" -> rating_overall=4.7, reviewer_count=123

    Parameters:
        df (pd.DataFrame): DataFrame chứa cột đánh giá.
        col_name (str): Tên cột chứa dữ liệu đánh giá, ví dụ 'bus_rating'.

    Returns:
        pd.DataFrame: Thêm 2 cột 'rating_overall' và 'reviewer_count'.
    """
    split_cols = df[col_name].str.split(' ', expand=True)
    df['rating_overall'] = pd.to_numeric(split_cols[0], errors='coerce')
    df['reviewer_count'] = split_cols[1].replace(r'[()]', '', regex=True).astype(int)
    return df

In [4]:
def extract_number_of_seats(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
    """
    Trích xuất số lượng ghế từ cột 'seat_type' và tạo cột mới 'number_of_seat'.

    Ví dụ:
        "Xe giường nằm 40 chỗ"  ->  40
        "Ghế ngồi 29 chỗ"       ->  29

    Parameters:
        df (pd.DataFrame): DataFrame chứa cột 'seat_type' mô tả loại ghế.

    Returns:
        pd.DataFrame: DataFrame đã được thêm cột 'number_of_seat'.
    """

    # Trích xuất số
    df['number_of_seat'] = df[col_name].replace(r'[^0-9]', '', regex=True)
    df['number_of_seat'] = pd.to_numeric(df['number_of_seat'], errors='coerce').astype(int)

    return df

In [5]:
def normalize_fare(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    """
    Chuẩn hoá giá vé về dạng số nguyên (VND).

    Ví dụ:
        "Từ 350.000đ" -> 350000
        "200.000đ"    -> 200000

    Parameters:
        df (pd.DataFrame): DataFrame chứa dữ liệu giá.
        cols (list[str]): Danh sách 2 cột ['original_price', 'sale_price'].

    Returns:
        pd.DataFrame: Thêm 2 cột mới 'price_original' và 'price_discounted'.
    """
    
    price = df[cols].replace(r'[^0-9]', '', regex=True)
    # fillna = 0 và ép kiểu sang int
    price = price.apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)   

    df['price_original'] = price.iloc[:, 0]
    df['price_discounted'] = price.iloc[:, 1]

    return df


In [6]:
def convert_to_time(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
    """
    Chuyển đổi cột thời gian từ chuỗi (string) sang kiểu datetime.time.

    Ví dụ:
        "07:30"  ->  "7:30:00"
        "21:05"  ->  "21:05:00"

    Parameters:
        df (pd.DataFrame): DataFrame chứa dữ liệu cần xử lý.
        col_name (str): Tên cột chứa dữ liệu thời gian (ví dụ 'departure_time').

    Returns:
        pd.DataFrame: DataFrame với cột đã được chuyển sang kiểu datetime.time.
    """

    df[col_name] = pd.to_datetime(df[col_name], format='%H:%M', errors='coerce').dt.time
    return df

In [7]:
def normalize_date(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
    """
    Chuẩn hóa cột ngày tháng từ chuỗi sang định dạng 'YYYY-MM-DD'.

    Ví dụ:
        "T4, 17/10/2025"  ->  "2025-10-17"

    Parameters:
        df (pd.DataFrame): DataFrame chứa dữ liệu cần xử lý.
        col_name (str): Tên cột chứa dữ liệu ngày tháng (ví dụ 'departure_date').

    Returns:
        pd.DataFrame: DataFrame với cột 'departure_time' đã được chuẩn hóa.
    """

    date = df[col_name].str.split(', ', expand=True)[1]
    df[col_name] = pd.to_datetime(date, format='%d/%m/%Y', errors='coerce').dt.strftime('%Y-%m-%d')
    return df


In [8]:
def normalize_duration(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
    """
    Chuẩn hóa cột thời lượng di chuyển sang tổng số phút (int).

    Ví dụ:
        "2h30m" -> 150
        "1h"    -> 60
        "45m"   -> 45

    Parameters:
        df (pd.DataFrame): DataFrame chứa dữ liệu cần xử lý.
        col_name (str): Tên cột chứa thời lượng di chuyển (ví dụ 'duration').

    Returns:
        pd.DataFrame: DataFrame với cột mới 'duration_m' biểu thị tổng số phút di chuyển.
    """

    # Trích xuất phần giờ và phút
    hours = df[col_name].str.extract(r'(\d+)h')[0].astype('Int64').fillna(0)
    mins = df[col_name].str.extract(r'(\d+)m')[0].astype('Int64').fillna(0)

    # Tính tổng số phút
    df['duration_m'] = hours * 60 + mins
    return df


In [9]:
def handle_bus_name(df: pd.DataFrame, column: str) -> pd.DataFrame:
    """
    Chuẩn hóa tên nhà xe bằng cách loại bỏ phần trong ngoặc.

    Ví dụ:
        "Hoàng Long (Ghế ngồi)" -> "Hoàng Long"

    Parameters:
        df (pd.DataFrame): DataFrame chứa dữ liệu cần xử lý.

    Returns:
        pd.DataFrame: DataFrame với cột 'bus_name' đã được làm sạch.
    """

    df[column].replace(r'\s*\([^)]+\)', '', regex=True, inplace=True)
    return df

In [10]:
def normalize_location_type(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
    """
    Chuẩn hóa loại địa điểm (ví dụ: 'Bến xe', 'Văn phòng', 'Other') 
    trong cột chỉ định của DataFrame bằng cách mở rộng các từ viết tắt thông dụng.

    Parameters:
        df (pd.DataFrame): DataFrame chứa dữ liệu cần xử lý.
        col_name (str): Tên cột chứa dữ liệu văn bản cần chuẩn hóa
                        (ví dụ: 'pick_up_point' hoặc 'drop_off_point').

    Returns:
        pd.DataFrame: DataFrame với cột đã được chuẩn hóa loại địa điểm.
    """
    new_col = f"{col_name}_type"
    df[new_col] = df[col_name].str.lower().apply(
        lambda x: "Bến xe" if "bx" in x or "bến xe" in x
        else "Văn phòng" if "vp" in x or "văn phòng" in x
        else "Other"
    )

    return df

In [1]:
def fill_missing_ratings(df: pd.DataFrame, cols: list[str], default_value: float = 3.5) -> pd.DataFrame:
    """
    Điền giá trị thiếu cho các cột đánh giá theo trung bình của từng nhà xe.
    Nếu nhà xe không có dữ liệu, dùng trung bình toàn cột để thay thế.

    Returns:
        pd.DataFrame: DataFrame với các cột đánh giá đã được chuẩn hóa.
    """

    df[cols] = (
        df.groupby('company_name', group_keys=False)[cols]
          .apply(lambda g: g.fillna(g.mean().round(1)))
          .fillna(df[cols].mean().round(1))
          .fillna(default_value)
    )
    return df

NameError: name 'pd' is not defined

In [ ]:
def drop_unused_columns(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    """
    Loại bỏ các cột không cần thiết khỏi DataFrame.

    Parameters:
        df (pd.DataFrame): DataFrame cần xử lý.
        columns_to_drop (list[str]): Danh sách tên cột cần xóa.

    Returns:
        pd.DataFrame: DataFrame sau khi loại bỏ các cột không cần thiết.
    """

    # Loại bỏ các cột đã được trích xuất dữ liệu hoàn thành, không dùng tới nữa
    df.drop(columns=cols, errors='ignore', inplace=True)

    # Loại bỏ cột có toàn bộ giá trị là None (nếu có)
    df.dropna(axis=1, how='all', inplace=True)

    # Lọc dữ liệu có giá vé hoặc số người đánh giá > 0
    df = df[(df['reviewer_count'] >= 0) | (df['price_original'] >= 0)]
    return df

In [13]:
def rename_cols(df: pd.DataFrame, dict_rename_cols: dict[str]):

    df.rename(columns=dict_rename_cols, inplace=True)
    return df

In [14]:
def preprocess_transport_data(df, dict_rename_cols, unused_cols, rating_cols):
    """ 
    Tiền xử lý toàn bộ dữ liệu xe khách:
    - Chuẩn hoá rating, giá, thời gian, tên nhà xe
    - Làm sạch điểm đón/trả
    - Điền giá trị thiếu cho rating theo trung bình của từng nhà xe
    """

    df = extract_rating_features(df, 'bus_rating')  # trích xuất phần đánh giá (sao)
    df = extract_number_of_seats(df, 'seat_type')   # trích xuất ra số lượng ghế
    df = normalize_fare(df, ['price_original', 'price_discounted'])   # chuẩn hoá, ép kiểu dữ liệu sang int
    df = convert_to_time(df, 'departure_time')  # chuẩn hoá thành hh:mm:00
    df = normalize_date(df, 'departure_date')   # chuẩn hoá thành yyyy-MM-DD
    df = normalize_duration(df, 'duration') # chuẩn hoá thành phút
    df = handle_bus_name(df, 'company_name')    # Làm sạch tên nhà xe
    df = normalize_location_type(df, 'pickup_point')
    df = normalize_location_type(df, 'dropoff_point')
    df = fill_missing_ratings(df, rating_cols)  # fillna()
    df = rename_cols(df, dict_rename_cols)
    df = drop_unused_columns(df, unused_cols)   # bỏ những cột không dùng đến/đã được trích xuất
    return df

---

In [15]:
dict_rename_cols = {
# ratings
    'An toàn': 'rating_safety',
    'Thông tin chính xác': 'rating_info_accuracy',
    'Thông tin đầy đủ': 'rating_info_completeness',
    'Thái độ nhân viên': 'rating_staff_attitude',
    'Tiện nghi & thoải mái': 'rating_comfort',
    'Chất lượng dịch vụ': 'rating_service_quanlity',
    'Đúng giờ': 'rating_punctuality',
# trips info
    'pick_up_point_type': 'pickup_point',
    'drop_of_point_type': 'dropoff_point'
}

rating_cols = ['An toàn', 'Thông tin chính xác', 'Thông tin đầy đủ','Thái độ nhân viên', 'Tiện nghi & thoải mái','Chất lượng dịch vụ', 'Đúng giờ']
unused_cols = ['arrival_time', 'arrival_date', 'bus_rating', 'seat_type', 'sale_price', 'original_price', 'percent_discount', 'duration', 'pickup_point', 'dropoff_point']

In [16]:
df = pd.read_csv('../../data/raw/Hà Nội_Hà Giang_5_11_2025.csv')
df

,company_name,bus_rating,seat_type,departure_time,pickup_point,arrival_date,arrival_time,dropoff_point,duration,price_original,...,departure_date,start_point,destination,An toàn,Thông tin chính xác,Thông tin đầy đủ,Thái độ nhân viên,Tiện nghi & thoải mái,Chất lượng dịch vụ,Đúng giờ
0,Bằng Phấn,4.6 (1709),Limousine 13 chỗ,07:00,• 156 Trần Quang Khải,NaN,13:30,• Bến Xe Hà Giang,6h30m,300.000đ,...,"T4, 05/11/2025",Hà Nội,Hà Giang,4.7,4.7,4.7,4.6,4.6,4.6,4.8
1,Quang Tuyến (Hà Giang),4.1 (257),Limousine giường phòng 24 chỗ,23:00,• 55 Nguyễn Hoàng,(06/11),05:30,• Văn phòng Hà Giang,6h30m,Từ 350.000đ,...,"T4, 05/11/2025",Hà Nội,Hà Giang,4.4,4.1,4.2,4.1,4.1,4.0,4.3
2,Mạnh Quân,4.5 (594),Cabin cung điện 24 phòng,14:30,• Bến xe Mỹ Đình (Ô đón khách số 46),NaN,21:00,• Văn Phòng 75 Cầu Mè (Gần Bến Xe Hà Giang đối...,6h30m,Từ 350.000đ,...,"T4, 05/11/2025",Hà Nội,Hà Giang,4.6,4.4,4.5,4.4,4.3,4.2,4.8
3,Vĩnh Thiện,4.3 (42),Limousine giường phòng 24 chỗ,11:00,• Bến xe Gia Lâm,NaN,18:30,• Bến xe Hà Giang,7h30m,Từ 350.000đ,...,"T4, 05/11/2025",Hà Nội,Hà Giang,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Bus Vip Limousine Ha Giang,4.8 (10),Limousine 9 chỗ,07:00,• Số 11 Hàng Rươi,NaN,13:00,"• Số 5 Đại Lộ Hữu Nghị, Hà Giang",6h,Từ 320.000đ,...,"T4, 05/11/2025",Hà Nội,Hà Giang,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Cầu Mè,4.3 (147),Giường nằm 41 chỗ,09:30,• Bến xe Mỹ Đình,NaN,16:30,• Cầu Mè Hotel,7h,250.000đ,...,"T4, 05/11/2025",Hà Nội,Hà Giang,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Đăng Quang (Hà Giang),5 (13),Limousine 24 chỗ,23:00,• 102 Trần Vỹ,(06/11),05:35,• Văn phòng Hà Giang,6h35m,350.000đ,...,"T4, 05/11/2025",Hà Nội,Hà Giang,4.8,4.7,4.7,4.7,4.7,4.8,4.7
7,Quang Nghị,3.9 (715),Giường nằm 46 chỗ,22:30,• Văn Phòng Mỹ Đình,(06/11),05:00,• Bến xe Hà Giang,6h30m,250.000đ,...,"T4, 05/11/2025",Hà Nội,Hà Giang,4.3,4.0,4.1,3.9,4.0,3.8,4.5
8,Hà Tuấn,4 (65),Limousine giường phòng 24 chỗ,12:30,• Văn phòng Hà Nội,NaN,19:00,• Văn phòng Hà Giang,6h30m,Từ 350.000đ,...,"T4, 05/11/2025",Hà Nội,Hà Giang,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Đông Bắc Travel,4.9 (38),Xe Limousine 16 chỗ,16:00,• Văn phòng Hà Nội,NaN,22:00,• 165 đường Lý Tự Trọng,6h,300.000đ,...,"T4, 05/11/2025",Hà Nội,Hà Giang,5.0,5.0,5.0,5.0,5.0,5.0,4.9


In [17]:
df = preprocess_transport_data(df, dict_rename_cols, unused_cols, rating_cols)
# ../data/raw/Sài Gòn_Vũng Tàu_1_11_2025.csv
# df.drop_duplicates(keep='first', inplace=True)

In [18]:
df.duplicated().sum()

0

In [20]:
df

,original_price,sale_price,price_original,price_discounted
0,Từ 350.000đ,300.000đ,350000,300000
1,200.000đ,180.000đ,200000,180000
2,400.000đ,350.000đ,400000,350000


In [21]:
df.isna().sum()

original_price      0
sale_price          0
price_original      0
price_discounted    0
dtype: int64

In [22]:
read_csv('../../data/raw/Sài Gòn_Gia Lai_5_11_2025.csv')

,company_name,bus_rating,seat_type,departure_time,pickup_point,arrival_date,arrival_time,dropoff_point,duration,price_original,...,departure_date,start_point,destination,An toàn,Thông tin chính xác,Thông tin đầy đủ,Thái độ nhân viên,Tiện nghi & thoải mái,Chất lượng dịch vụ,Đúng giờ
0,Hoàng Thuỷ,4.7 (3454),Limousine 34 Giường VIP,18:45,• Bến xe Miền Đông - Quầy vé 37,(06/11),05:45,• Bến Xe Đức Long Gia Lai,11h,350.000đ,...,"T4, 05/11/2025",Sài Gòn,Gia Lai,4.8,4.7,4.7,4.6,4.6,4.6,4.9
1,Sinh Diên Hồng,4.6 (236),Limousine 34 chỗ,17:30,• Bến xe An Sương,(06/11),04:15,• Bến Xe Đức Long Gia Lai,10h45m,370.000đ,...,"T4, 05/11/2025",Sài Gòn,Gia Lai,4.7,4.7,4.8,4.8,4.6,4.6,4.9
2,Đức Đạt,4.7 (1016),Limousine giường nằm 34 chỗ,20:10,• Bến xe Miền Đông - Quầy vé 17,(06/11),06:35,• Bến Xe Đức Long Gia Lai,10h25m,350.000đ,...,"T4, 05/11/2025",Sài Gòn,Gia Lai,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Kính Diên Hồng,4.6 (1132),Limousine giường nằm 34 chỗ,18:40,• Bến Xe Miền Đông - Quầy vé 19,(06/11),05:40,• Bến Xe Đức Long Gia Lai,11h,350.000đ,...,"T4, 05/11/2025",Sài Gòn,Gia Lai,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Vương Tấn Dũng,4.8 (382),Limousine 34 Phòng,16:45,• VP QL13,(06/11),06:00,• An Khê - Quốc Lộ 19,13h15m,380.000đ,...,"T4, 05/11/2025",Sài Gòn,Gia Lai,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Thuận Tiến,4.7 (4950),Limousine 34 giường,18:50,• Bến xe Miền Đông - Quầy vé 24,(06/11),07:15,• Bến Xe Đức Long Gia Lai,12h25m,350.000đ,...,"T4, 05/11/2025",Sài Gòn,Gia Lai,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Bảy Lang,4.7 (278),Limousine 34 giường,18:30,"• Bến xe miền đông cũ (Dãy số 6, Ô C14)",(06/11),05:30,• Bến Xe Đức Long Gia Lai,11h,350.000đ,...,"T4, 05/11/2025",Sài Gòn,Gia Lai,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Phong Phú,4.5 (8653),Limousine 24 phòng đôi,20:00,• Văn phòng Hoàng Văn Thụ.,(06/11),07:45,• VP Gia Lai,11h45m,Từ 530.000đ,...,"T4, 05/11/2025",Sài Gòn,Gia Lai,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,An Phát,4.5 (379),Limousine 34 phòng VIP,16:30,• BX Miền Đông - Cổng 4,(06/11),06:15,• Bến Xe K-Bang,13h45m,400.000đ,...,"T4, 05/11/2025",Sài Gòn,Gia Lai,4.6,4.5,4.6,4.5,4.3,4.5,4.8
9,Sáu Bản,4.4 (485),Giường nằm 44 chỗ,17:00,• Bến xe An Sương,(06/11),04:10,• PleiMe,11h10m,300.000đ,...,"T4, 05/11/2025",Sài Gòn,Gia Lai,NaN,NaN,NaN,NaN,NaN,NaN,NaN
